In [1]:
from reservoir_rls import *
import matplotlib.pyplot as plt
from lorenz63 import *
from scipy.signal import welch, periodogram

In [2]:
step = 0.05
start_time = 125
sync_length = 200
num_tests = 500
train_length = 800
pred_length = 400
base_data_length = int(start_time/step)
data_length = int(base_data_length * step/(0.01))+pred_length
r_t = lambda x: r_t_cosine(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)

num_tests = 100
with tqdm(total = num_tests) as pbar:
    for seed in range(num_tests):
        data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = seed)
        if seed == 0:
            optim_data = data[-sync_length - train_length - pred_length:]
        else:
            optim_data = np.vstack((optim_data, data[-sync_length - train_length - pred_length:]))
        pbar.update(1)
print(optim_data.shape)

# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
scaled_data = np.copy(optim_data)
scaled_data = np.ascontiguousarray(scaled_data)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:55<00:00,  1.81it/s]

(140000, 3)


In [4]:
num_nodes = 210
num_tests = 100
train_length = 800
sync_length = 200
pred_length = 400
res_seed = 1
input_weights = np.linspace(-1,1,25)
regularizations = np.logspace(-13,-3,11)
forgets = np.array([1,0.999,0.995,0.99,0.985,0.98,0.97])

vts = np.zeros(input_weights.size)
for i, weight in enumerate(input_weights):
    res1 = reservoir(3,num_nodes,input_weight = weight, seed = res_seed, forget = 1.)
    results = cross_validation_performance_separated(scaled_data, res1, num_tests, sync_length, train_length, pred_length, \
         errormax = 3.2, train_method = 'Normal', progress = False)
    vts[i] = np.median(results)
best_weight = input_weights[vts == np.max(vts)][0]
print(np.max(vts), best_weight)

vts = np.zeros(forgets.size)
for i, forget in enumerate(forgets):
    res1 = reservoir(3,num_nodes, input_weight = best_weight, \
                     forget = forget, seed = res_seed)
    results = cross_validation_performance_separated(scaled_data, res1, num_tests, sync_length, train_length, pred_length, \
         errormax = 3.2, train_method = 'Normal', progress = False)
    vts[i] = np.median(results)
best_forget = forgets[vts == np.max(vts)][0]
print(np.max(vts), best_forget)

vts = np.zeros(regularizations.size)
for i, regularization in enumerate(regularizations):
    res1 = reservoir(3,num_nodes, input_weight = best_weight, \
             regularization = regularization, seed = res_seed, forget = best_forget)
    results = cross_validation_performance_separated(scaled_data, res1, num_tests, sync_length, train_length, pred_length, \
         errormax = 3.2, train_method = 'Normal', progress = False)
    vts[i] = np.median(results)
best_regularization = regularizations[vts == np.max(vts)][0]
print(np.max(vts), best_regularization)

25.5 -0.08333333333333337
25.5 1.0
25.5 1e-06


In [ ]:
step = 0.05
start_time = 125
sync_length = 200
num_tests = 500
train_length = 800
pred_length = 400
seed = 25
base_data_length = int(start_time/step)
data_length = int(base_data_length * step/(0.01))+pred_length
r_t = lambda x: r_t_cosine(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)

num_tests = 100
with tqdm(total = num_tests) as pbar:
    for seed in range(num_tests):
        data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = seed)
        if seed == 0:
            optim_data = data[-sync_length - train_length - pred_length:]
        else:
            optim_data = np.vstack((optim_data, data[-sync_length - train_length - pred_length:]))
        pbar.update(1)
print(optim_data.shape)

# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
scaled_data = np.copy(optim_data)
scaled_data = np.ascontiguousarray(scaled_data)

In [12]:
res1 = reservoir(3,num_nodes, input_weight = best_weight, regularization = best_regularization, \
                     forget = best_forget, seed = res_seed)
results = cross_validation_performance_separated(scaled_data, res1, num_tests, sync_length, train_length, pred_length, \
     errormax = 3.2, train_method = 'Normal', progress = False)

print('Test Valid Time: %f' % np.median(results))

Test Valid Time: 35.000000
